# **Práctica Final: Final_Predicción de duración de viajes y detección de emergencias en tweets**

### **Parte 1: Predicción de duración de viajes**

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import warnings
import torch

from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PowerTransformer, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from transformers import AutoModel, AutoTokenizer

warnings.filterwarnings('ignore')
random_state = 100

### Ejercicio 1:
(2 puntos). Realizar preprocesamiento de datos: imputar valores faltantes, transformar variables categóricas, estandarizar variables numéricas, etc.
si lo consideras necesario para futuros modelos. Puede ser interesante intentar adaptar las variables que no siguan una distribución normal mediante técnicas de mapeado a gausianas como Power Transformers (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer).

Primero leeremos los dataframes para hacernos una idea de con que estamos trabajando.

In [ ]:
uberex = pd.read_csv("/content/drive/MyDrive/Data 4 segundo cuatri/Practica final/uber_time_examples.csv")
uberlab = pd.read_csv("/content/drive/MyDrive/Data 4 segundo cuatri/Practica final/uber_time_labels.csv")
uberex.shape, uberlab.shape

((400000, 12), (400000, 2))

Ambos frames tienen el mismo numero de filas y columnas.
Ahora utilizaremos un .head para ver como son.

In [ ]:
uberex.head(5), uberlab.head(5)

(   id       feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
 0   0  01-07 17:04:08          2       1.20        263        141  12.513054   
 1   1  03-02 17:41:40          1       0.88        246         68   6.256527   
 2   2  02-17 12:15:00          3       7.61         24         13  18.769581   
 3   3  03-30 13:59:42          1       1.50        239        163   6.256527   
 4   4  02-14 18:26:55          1       1.20        142        229   6.256527   
 
     feature_6  feature_7  feature_8  feature_9  feature_10  
 0  297.430685  56.317405     405.20   0.408689  126.689773  
 1  278.205127  27.160167     314.88  -0.256911  126.693467  
 2   27.141964   5.192385      44.61  56.880789  126.615789  
 3  270.288721  65.104518     403.50   1.218689  126.686311  
 4  160.589952  91.465857     372.20   0.408689  126.689773  ,
    id  duration
 0   0     455.0
 1   1     413.0
 2   2    1501.0
 3   3     514.0
 4   4     605.0)

Obeservamos que la primera columna es el "id" los que se usa para identificar, esto nos faicilitara el trabjao a la hora de juntar las tablar que es lo que haremos ahora con un .merge

In [ ]:
data = pd.merge(uberex, uberlab, on= "id")
data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration
0,0,01-07 17:04:08,2,1.20,263,141,12.513054,297.430685,56.317405,405.20,0.408689,126.689773,455.0
1,1,03-02 17:41:40,1,0.88,246,68,6.256527,278.205127,27.160167,314.88,-0.256911,126.693467,413.0
2,2,02-17 12:15:00,3,7.61,24,13,18.769581,27.141964,5.192385,44.61,56.880789,126.615789,1501.0
3,3,03-30 13:59:42,1,1.50,239,163,6.256527,270.288721,65.104518,403.50,1.218689,126.686311,514.0
4,4,02-14 18:26:55,1,1.20,142,229,6.256527,160.589952,91.465857,372.20,0.408689,126.689773,605.0


Ahora ya que tenemos la tabla lsita para empezar a trabaja, primero haremos un preanalisis de preprocesamiento de datos, tratanto los valores nulos, tipos de variabales, si es necesario tranformarlas, etc, como hemos hecho en otras ocasiones.

Primero trataremos los valores nulos

In [ ]:
data.isnull().sum()

id            0
feature_0     0
feature_1     0
feature_2     0
feature_3     0
feature_4     0
feature_5     0
feature_6     0
feature_7     0
feature_8     0
feature_9     0
feature_10    0
duration      0
dtype: int64

No hay datos nan en todo el conjunto.

In [ ]:
data.dtypes

id              int64
feature_0      object
feature_1       int64
feature_2     float64
feature_3       int64
feature_4       int64
feature_5     float64
feature_6     float64
feature_7     float64
feature_8     float64
feature_9     float64
feature_10    float64
duration      float64
dtype: object

La unica variable que nos conviene tranformar es la "feature_0" ya que son fechas y comviene que no sea de tipo objedct.

In [ ]:
data['feature_0'] = pd.to_datetime(data['feature_0'], format='%m-%d %H:%M:%S')
data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration
0,0,1900-01-07 17:04:08,2,1.20,263,141,12.513054,297.430685,56.317405,405.20,0.408689,126.689773,455.0
1,1,1900-03-02 17:41:40,1,0.88,246,68,6.256527,278.205127,27.160167,314.88,-0.256911,126.693467,413.0
2,2,1900-02-17 12:15:00,3,7.61,24,13,18.769581,27.141964,5.192385,44.61,56.880789,126.615789,1501.0
3,3,1900-03-30 13:59:42,1,1.50,239,163,6.256527,270.288721,65.104518,403.50,1.218689,126.686311,514.0
4,4,1900-02-14 18:26:55,1,1.20,142,229,6.256527,160.589952,91.465857,372.20,0.408689,126.689773,605.0


Ahora utilizando el metodo que nos dan en el enunciado, transformaremos la variable udracion, la cual no sigue una distribucion normal.
Metodo Yeo-Johnson

In [ ]:
transformer = PowerTransformer(method= "yeo-johnson")
data["duration"] = transformer.fit_transform(data[["duration"]])
data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration
0,0,1900-01-07 17:04:08,2,1.20,263,141,12.513054,297.430685,56.317405,405.20,0.408689,126.689773,-0.378044
1,1,1900-03-02 17:41:40,1,0.88,246,68,6.256527,278.205127,27.160167,314.88,-0.256911,126.693467,-0.490657
2,2,1900-02-17 12:15:00,3,7.61,24,13,18.769581,27.141964,5.192385,44.61,56.880789,126.615789,1.083267
3,3,1900-03-30 13:59:42,1,1.50,239,163,6.256527,270.288721,65.104518,403.50,1.218689,126.686311,-0.235028
4,4,1900-02-14 18:26:55,1,1.20,142,229,6.256527,160.589952,91.465857,372.20,0.408689,126.689773,-0.041645


### Ejercicio 2.
(2 punto) Crear nuevas características (features) que puedan mejorar el poder predictivo del modelo.

Tenemos una columna en formato tiempo, y puede ser interesante crear tres diferentes en base a esta para tener el dia, mes y hora de cada reserva de hora. Así podremos ver la evolución a lo largo de los meses o si en algún dia/a alguna hora los trayectos son más largos.



In [ ]:
data['day'] = data['feature_0'].dt.day
data['month'] = data['feature_0'].dt.month
data['hour'] = data['feature_0'].dt.hour

Elimino la columna de las fechas xq creo que ya no aporta info. Tambien la de id porque tampoco nos aporta mucho.

In [ ]:
data = data.drop(['feature_0'], axis=1)
data = data.drop(['id'], axis = 1)
data.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration,day,month,hour
0,2,1.20,263,141,12.513054,297.430685,56.317405,405.20,0.408689,126.689773,-0.378044,7,1,17
1,1,0.88,246,68,6.256527,278.205127,27.160167,314.88,-0.256911,126.693467,-0.490657,2,3,17
2,3,7.61,24,13,18.769581,27.141964,5.192385,44.61,56.880789,126.615789,1.083267,17,2,12
3,1,1.50,239,163,6.256527,270.288721,65.104518,403.50,1.218689,126.686311,-0.235028,30,3,13
4,1,1.20,142,229,6.256527,160.589952,91.465857,372.20,0.408689,126.689773,-0.041645,14,2,18


In [ ]:
data.nunique()

feature_1        10
feature_2      2709
feature_3       231
feature_4       255
feature_5        10
feature_6       231
feature_7       255
feature_8     40197
feature_9      2709
feature_10     2709
duration       5592
day              31
month             4
hour             24
dtype: int64

Aplicaremos un getdummies para la columna "month" ya que solo toma 4 variables diferentes.

In [ ]:
data = pd.get_dummies(data, columns = ["month"])
data.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration,day,hour,month_1,month_2,month_3,month_12
0,2,1.20,263,141,12.513054,297.430685,56.317405,405.20,0.408689,126.689773,-0.378044,7,17,True,False,False,False
1,1,0.88,246,68,6.256527,278.205127,27.160167,314.88,-0.256911,126.693467,-0.490657,2,17,False,False,True,False
2,3,7.61,24,13,18.769581,27.141964,5.192385,44.61,56.880789,126.615789,1.083267,17,12,False,True,False,False
3,1,1.50,239,163,6.256527,270.288721,65.104518,403.50,1.218689,126.686311,-0.235028,30,13,False,False,True,False
4,1,1.20,142,229,6.256527,160.589952,91.465857,372.20,0.408689,126.689773,-0.041645,14,18,False,True,False,False


### Ejercicio 3.
(2 puntos) Seleccionar las características más relevantes para predecir la duración del viaje. Utilizar técnicas de selección de características basadas en una sola variable o SelectFromModel. Evitar Recursive feature elimination debido a su alto coste computacional.
Ver : (https://scikit-learn.org/stable/modules/feature_selection.html)

Utilizaremos la tecnica de seleccion vasado en correlacion.

In [ ]:
corr_mtx = data.corr()
corr_with_target = corr_mtx["duration"].abs().sort_values(ascending=False)
corr_with_target

duration      1.000000
feature_2     0.551888
feature_10    0.551888
feature_7     0.080211
feature_4     0.080211
feature_8     0.071753
feature_6     0.058250
feature_3     0.058250
hour          0.040001
feature_1     0.018312
feature_5     0.018312
feature_9     0.016372
day           0.014297
month_3       0.012911
month_1       0.011181
month_12      0.007903
month_2       0.002041
Name: duration, dtype: float64

Ahora utilizaremos un SelectFromModel para comprobar y ver con cual nos quedamos, ya que una variable puede estar muy correlacionada con la objetivo de forma negativa y no daría como resultado que están altamente relacionadas cuando si que lo están.
Lo haremos con un modelos de clasificacion basado en arboles

In [ ]:
x = data.drop(["duration"], axis = 1)
y = data["duration"]

model = RandomForestRegressor()
model.fit(x,y)

features = model.feature_importances_


Ahora crearemos un diccionario para visualizarlo y imprimeremps de forma descendeinte en importancia

In [ ]:
dic = dict(zip(x.columns, features))
for char, importance in sorted(dic.items(), key = lambda x: x[1], reverse = True):
  print(f"{char}: {importance}")

feature_10: 0.37620238537638967
feature_9: 0.16392194952438285
feature_2: 0.1559376906784803
feature_8: 0.06419665741841121
hour: 0.056859757236187386
day: 0.051872273004342245
feature_7: 0.02467333971566845
feature_4: 0.024245257412212718
feature_6: 0.022151972735875925
feature_3: 0.02181284869296713
feature_5: 0.008770179401805172
feature_1: 0.00873613728613471
month_1: 0.007127760902477079
month_3: 0.0067684991096385715
month_2: 0.0065095444181009175
month_12: 0.000213747086925675


El resultado que tenemos es ligeramente diferente, así que nos quedamos con el de SelecFromModel para concluir que las características más importantes en relación a la variable objetivo son: feature_10, feature_2 y feature_9

### Ejercicio 4
(2 puntos) Entrenar un modelo sencillo como base y medir su MAPE (Mean Absolute Percentage Error) en el conjunto de test. Luego, elegir y entrenar dos modelos más avanzados (por ejemplo, ensambladores, máquinas de soporte vectorial, modelos bayesianos, redes neuronales) y comparar sus MAPEs.

Ya que para el random forest nos ha tardado un poco, para este haremos una muestra de 15000 filas para poder trabajar mas rapido, ya que la original es muy grande

In [ ]:
sample = data.sample(n=15000, random_state = random_state)


En primer lugar, dividimos nuestro conjunto de datos en dos dataframes, uno que contiene todas las variables excepto la objetivo y otro solo con la variable objetivo. Dividimos ambos en conjuntos de test y entrenamiento

In [ ]:
x = sample.drop(["duration"], axis = 1)
y = sample["duration"]


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=random_state)


Entrenaremos el modelo utilizando regresion lineal

In [ ]:
base_model = LinearRegression()
base_model.fit(x_train, y_train)

base_predict = base_model.predict(x_test)

y_pred_inv = transformer.inverse_transform(base_predict.reshape(-1, 1))
y_test_inv = transformer.inverse_transform(y_test.values.reshape(-1, 1))

# Calcular el error al cuadrado
squared_error = (y_pred_inv - y_test_inv) ** 2

out = pd.DataFrame({'Duración Real': y_test_inv.flatten(),
                            'Duración Predicha': y_pred_inv.flatten(),
                            'Squared Error': squared_error.flatten()})

print(out)

      Duración Real  Duración Predicha  Squared Error
0             952.0         706.655256   60194.043531
1             746.0         516.713438   52572.327734
2             901.0         883.063743     321.709326
3             547.0         551.897340      23.983938
4            1097.0        1121.009504     576.456263
...             ...                ...            ...
4495          890.0         942.851269    2793.256611
4496          539.0         527.595929     130.052840
4497          898.0         562.157444  112790.222482
4498          762.0         682.738952    6282.313801
4499          595.0         589.938950      25.614223

[4500 rows x 3 columns]


In [ ]:
mse_lr = mean_squared_error(out['Duración Real'], out['Duración Predicha'], squared=False)

Calculamos el MAPE


In [ ]:
mape_lr = mean_absolute_percentage_error(transformador.inverse_transform(y_test_inv), y_pred_inv)
print(f"MAPE del modelo base: {mape_lr}")

MAPE del modelo base: 2701942153722850.5


Ahora como modelos mas avanzados utilizaremos el random forest, SVR y MLP
Los entrenamos utilizando el mismo metodo que acabamos de utilizar

In [ ]:
#random forest
model_rf = RandomForestRegressor()
model_rf.fit(x_train, y_train)

predict_rf = model_rf.predict(x_test)

y_pred_inv_rf = transformer.inverse_transform(predict_rf.reshape(-1, 1))
y_test_inv_rf = transformer.inverse_transform(y_test.values.reshape(-1, 1))

# Calcular el error al cuadrado
squared_error = (y_pred_inv_rf - y_test_inv_rf) ** 2

out = pd.DataFrame({'Duración Real': y_test_inv_rf.flatten(),
                            'Duración Predicha': y_pred_inv_rf.flatten(),
                            'Squared Error': squared_error.flatten()})

print(out)

mse_rf = mean_squared_error(out['Duración Real'], out['Duración Predicha'], squared=False)

mape_rf = mean_absolute_percentage_error(transformador.inverse_transform(y_test_inv_rf), y_pred_inv_rf)
print(f"MAPE del modelo RF: {mape_rf}")

      Duración Real  Duración Predicha  Squared Error
0             952.0         829.802352   14932.265185
1             746.0         675.262876    5003.740714
2             901.0         973.253864    5220.620874
3             547.0         533.864837     172.532497
4            1097.0         871.674335   50771.655431
...             ...                ...            ...
4495          890.0        1384.548440  244578.159490
4496          539.0         747.098756   43305.092258
4497          898.0         834.594313    4020.281143
4498          762.0         631.984884   16903.930360
4499          595.0         864.744764   72762.237718

[4500 rows x 3 columns]
MAPE del modelo RF: 422860163731582.56


In [ ]:
#SVR

from sklearn.svm import SVR
model_svr = SVR(kernel='rbf', C=100, gamma=0.2, epsilon=1.1, max_iter=50000)
model_svr.fit(x_train, y_train)

predict_svr = model_svr.predict(x_test)

y_pred_inv_svr = transformer.inverse_transform(predict_svr.reshape(-1, 1))
y_test_inv_svr = transformer.inverse_transform(y_test.values.reshape(-1, 1))

# Calcular el error al cuadrado
squared_error = (y_pred_inv_svr - y_test_inv_svr) ** 2

out = pd.DataFrame({'Duración Real': y_test_inv_svr.flatten(),
                            'Duración Predicha': y_pred_inv_svr.flatten(),
                            'Squared Error': squared_error.flatten()})

print(out)

mse_svr = mean_squared_error(out['Duración Real'], out['Duración Predicha'], squared=False)

mape_svr = mean_absolute_percentage_error(transformador.inverse_transform(y_test_inv_svr), y_pred_inv_svr)
print(f"MAPE del modelo SVR: {mape_svr}")

      Duración Real  Duración Predicha  Squared Error
0             952.0         633.135738  101674.417342
1             746.0         633.135799   12738.327940
2             901.0         633.135738   71751.262656
3             547.0         633.135176    7419.268467
4            1097.0         633.135738  215170.053213
...             ...                ...            ...
4495          890.0         633.135738   65979.248900
4496          539.0         633.025045    8840.709072
4497          898.0         633.135738   70153.077086
4498          762.0         633.135738   16605.997929
4499          595.0         633.135731    1454.333983

[4500 rows x 3 columns]
MAPE del modelo SVR: 5069063902541489.0


In [ ]:
#MLP
model_mlp = MLPRegressor()
model_mlp.fit(x_train, y_train)

predict_mlp = model_mlp.predict(x_test)

y_pred_inv_mlp = transformer.inverse_transform(predict_mlp.reshape(-1, 1))
y_test_inv_mlp = transformer.inverse_transform(y_test.values.reshape(-1, 1))

# Calcular el error al cuadrado
squared_error = (y_pred_inv_mlp - y_test_inv_mlp) ** 2

out = pd.DataFrame({'Duración Real': y_test_inv_mlp.flatten(),
                            'Duración Predicha': y_pred_inv_mlp.flatten(),
                            'Squared Error': squared_error.flatten()})

print(out)

mse_mlp = mean_squared_error(out['Duración Real'], out['Duración Predicha'], squared=False)

mape_mlp = mean_absolute_percentage_error(transformador.inverse_transform(y_test_inv_mlp), y_pred_inv_mlp)
print(f"MAPE del modelo MLP: {mape_mlp}")

      Duración Real  Duración Predicha  Squared Error
0             952.0         660.498639   84973.043390
1             746.0         623.449200   15018.698468
2             901.0         624.693801   76345.115350
3             547.0         371.189323   30909.393980
4            1097.0        1039.664793    3287.325948
...             ...                ...            ...
4495          890.0         996.922627   11432.448228
4496          539.0         395.219932   20672.707901
4497          898.0         343.069604  307947.744420
4498          762.0         641.400754   14544.178223
4499          595.0         408.087406   34936.317849

[4500 rows x 3 columns]
MAPE del modelo MLP: 1935217320047212.2


Ponemos los resultados en una tabla para poder visulizar mejor las similitudes y diferencias y asi hacer un analisis

In [ ]:
models = ['Linear Regression', 'Random Forest', 'SVM', 'MLP']
mse = [mse_lr, mse_rf, mse_svr, mse_mlp]
mapes = [mape_lr.round(4), mape_rf.round(4), mape_svr.round(4), mape_mlp.round(4)]
summary_df = pd.DataFrame({'Model':models, 'MSE':mse, 'MAPE':mapes})
summary_df.head()

,Model,MSE,MAPE
0,Linear Regression,4057.523329,2.701942e+15
1,Random Forest,4056.147215,4.228602e+14
2,SVM,4110.419705,5.069064e+15
3,MLP,4111.381123,1.935217e+15


La columna MSE muestra el error cuadrático medio obtenido por cada modelo. El error cuadrático medio es una métrica comúnmente utilizada para evaluar la precisión de los modelos de regresión. Cuanto menor sea el valor del MSE, mejor será el ajuste del modelo a los datos, en este caso el modelo que da mejor resultados en base al MSE es Random Forest.

La columna MAPE muestra el error porcentual absoluto medio obtenido por cada modelo. El error porcentual absoluto medio es otra métrica de evaluación común para los modelos de regresión. Cuanto menor sea el valor del MAPE, mejor será el ajuste del modelo a los datos. Teniendo en cuenta esta métrica el modelo más acertado es SVM.

### Ejercicio 5
(2 puntos) Optimizar los hiperparámetros de los dos últimos modelos utilizando validación cruzada (cross-validation) y comparar sus MAPEs. Elegir el mejor modelo basándose en estos resultados.

creamos dataframes apartir del sample y dividimos en subconjuntos de train y test

In [ ]:
x = sample[["duration"]]
y = sample["duration"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42)

Escalamos datos de vqriables predictoras

In [ ]:
scaler = MinMaxScaler()
x_traim_sld = scaler.fit_transform(x_train)
x_test_sld = scaler.fit_transform(x_test)

Lo aplicamos, en vez de a los dos ultimos modelos a todos los que hemos utilizado en el ejericio anterior, para obtener un resultado mas exacto.

In [ ]:
#RL
lr = LinearRegression()
lr.fit(x_traim_sld, y_train)

#RF
rf = RandomForestRegressor()
rf.fit(x_traim_sld, y_train)

#SVR
svr = SVR()
svr.fit(x_traim_sld, y_train)

#MLP
mlp = MLPRegressor()
mlp.fit(x_traim_sld, y_train)


MLPRegressor()

Definimos hiperparametros

In [ ]:
linear_params = {}
random_forest_params = {'n_estimators': [10, 50, 100], 'max_depth': [None, 5, 10]}
svr_params = {'C': [1, 10, 100], 'gamma': [0.1, 0.01, 0.001]}
mlp_params = {'hidden_layer_sizes': [(10,), (50,), (100,)], 'activation': ['relu', 'tanh'], 'alpha': [0.0001, 0.001, 0.01]}

Realizamos la búsqueda de hiperparámetros con validación cruzada "GridSearchCV"

In [ ]:
lr_cv = GridSearchCV(lr, linear_params, cv=5)
lr_cv.fit(x_traim_sld, y_train)

rf_cv = GridSearchCV(rf, random_forest_params, cv=5)
rf_cv.fit(x_traim_sld, y_train)

svr_cv = GridSearchCV(svr, svr_params, cv=5)
svr_cv.fit(x_traim_sld, y_train)

mlp_cv = GridSearchCV(mlp, mlp_params, cv=5)
mlp_cv.fit(x_traim_sld, y_train)

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu', 'tanh'],
                         'alpha': [0.0001, 0.001, 0.01],
                         'hidden_layer_sizes': [(10,), (50,), (100,)]})

Mejores modelos con los hiperparametros optimizados y realizamos predicciones con modelos optimizados.

In [ ]:
best_lr = lr_cv.best_estimator_
best_rf = rf_cv.best_estimator_
best_svr = svr_cv.best_estimator_
best_mlp = mlp_cv.best_estimator_

linear_predictions = best_lr.predict(x_test_sld)
random_forest_predictions = best_rf.predict(x_test_sld)
svm_predictions = best_svr.predict(x_test_sld)
mlp_predictions = best_mlp.predict(x_test_sld)

Calculamos el MSE y el MAPE para todos los modelos y, una vez mas, los poenmos en una tabla para ayudar a la visualizacion de resultados

In [ ]:
linear_mse = mean_squared_error(y_test, linear_predictions)
linear_mape = mean_absolute_percentage_error(y_test, linear_predictions)

random_forest_mse = mean_squared_error(y_test, random_forest_predictions)
random_forest_mape = mean_absolute_percentage_error(y_test, random_forest_predictions)

svm_mse = mean_squared_error(y_test, svm_predictions)
svm_mape = mean_absolute_percentage_error(y_test, svm_predictions)

mlp_mse = mean_squared_error(y_test, mlp_predictions)
mlp_mape = mean_absolute_percentage_error(y_test, mlp_predictions)


results = pd.DataFrame({'Modelo Predictor': ['Regresión Lineal', 'Random Forest', 'SVM', 'MLP'],
    'MSE': [linear_mse.round(4), random_forest_mse.round(4), svm_mse, mlp_mse.round(4)],
    'MAPE': [linear_mape.round(4), random_forest_mape.round(4), svm_mape, mlp_mape.round(4)]})

print(results)

   Modelo Predictor       MSE     MAPE
0  Regresión Lineal  0.000000  0.00510
1     Random Forest  0.000000  0.00090
2               SVM  0.000089  0.03149
3               MLP  0.000100  0.01380


### **Parte 2: COMPLEMENTARIA, detección de emergencias en tweets**

Leemos el dataframe de tweets y hacemos lo mismo que con la del ejercicio 1 (nan, varaibles etc)


In [ ]:
twt = pd.read_csv("/content/drive/MyDrive/Data 4 segundo cuatri/Practica final/twitter_emergency.csv")
twt.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
twt.isnull().sum(), twt.shape


(id             0
 keyword       61
 location    2533
 text           0
 target         0
 dtype: int64,
 (7613, 5))

Sustituimos los nan por unknown

Utilizamos el modelo sentence tranformer de HuggingFace para obtener los embeddings, ya que por ejemplo connun robarte-base, obtendriamos un vector para cada uno de las palabras del texto

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(twt['text'].tolist())

# Add the embeddings to the dataframe as a new column
twt['embeddings'] = embeddings.tolist()

# Save the updated dataframe to a new CSV file
twt.to_csv("/content/drive/MyDrive/Data 4 segundo cuatri/Practica final/embeddings.csv")
print(embeddings.shape)

(7613, 384)


### 2.
(1,5 puntos) Crear y entrenar una pequeña red neuronal que utilice los embeddings, la palabra clave
(keyword) y la ubicación (location) para predecir si un tweet está relacionado con una emergencia o no. Gestionar los valores faltantes y agrupar las variables categóricas de manera adecuada.
No es necesario realizar una optimización de hiperparámetros exhaustiva, pero se pueden realizar ajustes si se desea.

NOTA: Si no se ha podido calcular los embeddings del ejercicio anterior, usar los que aparecen guardados como numpy.array en el fichero .

In [ ]:
!pip uninstall tensorflow


Found existing installation: tensorflow 2.16.1
Uninstalling tensorflow-2.16.1:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.16.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [ ]:
# Codificar las variables categóricas
encoder = LabelEncoder()
twt['keyword'] = encoder.fit_transform(twt['keyword'])
twt['location'] = encoder.fit_transform(twt['location'])

# Preparar los datos de entrada y salida
x = np.concatenate([embeddings, twt[['keyword', 'location']].values], axis=1)
y = twt['target'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Escalar características
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

from scikeras.wrappers import KerasClassifier

# Crear una función para construir el modelo de red neuronal
def create_model(optimizer_name):
    model = Sequential()
    model.add(Input(shape=x_train_scaled.shape[1:]))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=0.001)
    else:
        optimizer = 'rmsprop'

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Crear el modelo utilizando KerasClassifier para ser compatible con GridSearchCV
model = KerasClassifier(build_fn=create_model)

# Definir los hiperparámetros a ajustar
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 20],
    'optimizer_name': ['adam', 'rmsprop']
}

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(x_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_result.best_params_

# Crear y entrenar el modelo con los mejores hiperparámetros
best_model = create_model(best_params['optimizer_name'])
best_model.fit(x_train_scaled, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'])

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(x_test_scaled)

mse = mean_squared_error(y_test, y_pred)
print("MSE del modelo: %.2f" % mse)

# Calcular el error absoluto medio (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("MAE del modelo: %.2f" % mae)